In [7]:
import polars as pl
from fitparse import FitFile

import pathlib
import os

this_dir = pathlib.Path('.').parent.resolve()
data_dir = this_dir / 'data'

In [8]:
workouts = [data_dir / file for file in os.listdir(data_dir) if file.endswith('.fit')]

In [9]:
test = workouts[0]

In [12]:
file = FitFile(str(test))

In [ ]:
for i, x in enumerate(file.get_messages('record')):
    for y in x:
        print(f"{y.name}: {y.value} {y.units}")

altitude: 311.6 m
distance: 0.0 m
enhanced_altitude: 311.6 m
gps_accuracy: 1 m
position_lat: 495498720 semicircles
position_long: -972042341 semicircles
timestamp: 2018-11-21 23:12:32 None
altitude: 311.6 m
distance: 1.84 m
enhanced_altitude: 311.6 m
gps_accuracy: 1 m
position_lat: 495498815 semicircles
position_long: -972042281 semicircles
timestamp: 2018-11-21 23:12:33 None
altitude: 311.6 m
distance: 3.67 m
enhanced_altitude: 311.6 m
enhanced_speed: 0.592 m/s
gps_accuracy: 1 m
position_lat: 495498909 semicircles
position_long: -972042218 semicircles
timestamp: 2018-11-21 23:12:34 None
altitude: 311.4 m
distance: 4.63 m
enhanced_altitude: 311.4 m
enhanced_speed: 0.592 m/s
gps_accuracy: 1 m
position_lat: 495499002 semicircles
position_long: -972042148 semicircles
timestamp: 2018-11-21 23:12:35 None
altitude: 311.4 m
distance: 5.22 m
enhanced_altitude: 311.4 m
enhanced_speed: 0.592 m/s
gps_accuracy: 1 m
position_lat: 495499094 semicircles
position_long: -972042068 semicircles
timestamp